# Deps

In [2]:
""" Dependencies import """

# !pip install pytube
# !pip install moviepy
# !pip install pydub


import os
import numpy as np

import librosa

import pydub
import moviepy.editor as mvd
from pytube import YouTube, Playlist

import cv2 as cv
from cv2 import CAP_PROP_FPS as _cv_frame_rate
from cv2 import CAP_PROP_FRAME_WIDTH as _cv_frame_width
from cv2 import CAP_PROP_FRAME_HEIGHT as _cv_frame_height
from cv2 import CAP_PROP_FRAME_COUNT as _cv_frame_count
from cv2 import CAP_PROP_POS_FRAMES as _cv_current_frame_index

import skimage.io
from skimage.io import imread

from matplotlib import pyplot as plt

AttributeError: 'numpy._ArrayFunctionDispatcher' object has no attribute '__code__'

# Download playlist and video

In [ ]:
class YTRes():
    low_resolutions = ['360p', '240p', '144p']
    mid_resolutions = ['720p', '480p', '360p']
    high_resolution = ['1080p', '720p']

def download_video(video: str or YouTube, resolution: None or str or list = None):
    if type(video) is str:
        youtubeObject = YouTube(video)
    else:
        youtubeObject = video
    
    if not resolution:
        youtubeObject = youtubeObject.streams.get_highest_resolution()
    elif type(resolution) is list:
        for res in resolution:
            yto = youtubeObject.streams.filter(res=res).first
            if yto != None:
                youtubeObject = yto
                break
    else:
        youtubeObject.streams.filter(res=resolution).first()
        if not youtubeObject:
            print(f'Resolution {res} not avaliable')

    try:
        youtubeObject.download()
        print(f"End download {youtubeObject.title}")
    except:
        print("An error has occurred")
    

def download_playlist(playlist: str or Playlist, resolution: None or str or list = None):
    if type(playlist) is str:
        playlitsObject = Playlist(playlist)
    else:
        playlitsObject = playlist
    
    for iv,video in enumerate(playlitsObject.videos):
        print(f'{iv}. --- Start download {video.title}')
        download_video(video)

In [4]:
video_link = [
    'https://youtu.be/D5veVBWNxYw?list=PLBrn4fh0NHXk28UG7iyYSwfSQY-7a_A-R',
    'https://youtu.be/BuoXuXROH6I',
    'https://youtu.be/nXVBtg5rlYI'
][2]

playlist_link = [
    'https://www.youtube.com/playlist?list=PLBrn4fh0NHXk28UG7iyYSwfSQY-7a_A-R', # silly simphonies skeletons
][0]

resolution = YTRes().low_resolutions

In [5]:
download_video(video_link, resolution=None)
# download_playlist(playlist_link, resolution=resolution)

End download La Linea 132


# Handle video

In [5]:
def video_from_folder(folder: str, fullpath=True, ext: str or list = '.mp4'):
    video_files = []
    ext = list(ext) if type(ext) is str else ext
    _ = [video_files.extend(files_from_folder(folder, fullpath, e)) for e in ext]
    return sorted(video_files)

def audio_from_folder(folder: str, fullpath=True, ext: str or list = '.mp3'):
    audio_files = []
    ext = list(ext) if type(ext) is str else ext
    _ = [audio_files.extend(files_from_folder(folder, fullpath, e)) for e in ext]
    return sorted(audio_files)

def image_from_folder(folder: str, fullpath=True, ext: str or list = ['.jpg', '.png']):
    image_files = []
    ext = list(ext) if type(ext) is str else ext
    _ = [image_files.extend(files_from_folder(folder, fullpath, e)) for e in ext]
    return sorted(image_files)

def files_from_folder(folder: str, fullpath=True, ext: None or str = None):
    return sorted([f'{folder if fullpath else ""}{f}' for f in os.listdir(folder) if (f.endswith(ext) if ext else True)])

def read_video(filename: str):
    # return cv.VideoCapture(filename)
    return mvd.VideoFileClip(filename)

def video_write_audio(video_filename: str, mp3_filename: None or str = None):
    video = read_video(video_filename)
    mp3_filename = f'{video_filename[ : video_filename.index(".")]}.mp3' if mp3_filename==None else mp3_filename
    video.audio.write_audiofile(mp3_filename)

## Extract audio

In [7]:
""" From entire folder """

vpath = 'VideoMP4/LaLinea/'

videopaths = video_from_folder(vpath)

_ = [video_write_audio(vp) for vp in videopaths]

chunk:   4%|▎         | 125/3552 [00:00<00:02, 1242.69it/s, now=None]

MoviePy - Writing audio in VideoMP4/LaLinea/La Linea 132.mp3


MoviePy - Done.


In [10]:
""" From single file """

video_write_audio('VideoMP4/LaLinea/La Linea 132.mp4')

chunk:   4%|▍         | 142/3552 [00:00<00:02, 1417.69it/s, now=None]

MoviePy - Writing audio in VideoMP4/LaLinea/La Linea 132.mp3


MoviePy - Done.


## Split Audio by Frames 

In [ ]:
def files_couple(folder: str, fullpath: bool = True, ext_video: str or list = '.mp4', ext_audio: str or list ='.mp3'):
    video_files = video_from_folder(folder, fullpath=fullpath, ext=ext_video)
    audio_files = audio_from_folder(folder, fullpath=fullpath, ext=ext_audio)
    assert(len(video_files)==len(audio_files))
    return [[vf, af] for vf,af in zip(video_files, audio_files)]

def frame_by_index(video: cv.VideoCapture, frame_index: int):
    assert(frame_index >= 0 and frame_index <= video.get(_cv_frame_count))
    video.set(_cv_current_frame_index, frame_index)
    return video.read()[1]

def write_mp3(filename: str, rate: int, audio: np.array):
    channels = 2 if (audio.ndim == 2 and audio.shape[1] == 2) else 1
    y = np.int16(audio)
    song = pydub.AudioSegment(y.tobytes(), frame_rate=rate, sample_width=2, channels=channels)
    song.export(filename, format="mp3", bitrate="320k")

def frame_sounds_couple(videofile: str, audiofile: str, folder_name: str or None = None, out_name_prefix: str or None = None):
    video = cv.VideoCapture(videofile)
    video_rate = video.get(_cv_frame_rate)
    n_frames = int(video.get(_cv_frame_count))
    audio, audio_rate = librosa.load(audiofile)
    sample_per_frame = audio_rate / video_rate

    folder_name = folder_name if folder_name != None else videofile[:videofile.index('.')]
    out_name_prefix = out_name_prefix if out_name_prefix != None else videofile[videofile.rindex('/')+1 if '/' in videofile else 0 : videofile.index('.')]
    
    os.mkdir(folder_name)

    for fi in range(1, n_frames):
        frame = frame_by_index(video, fi)
        w_start, w_end = int((fi-1)*sample_per_frame), int(fi*sample_per_frame)
        audio_interval = audio[w_start : w_end]
        
        filename = f'{folder_name}{"" if folder_name.endswith("/") else "/"}{out_name_prefix}_seg_{str(fi).rjust(6, "0")}'
        frame_filename, audio_filename = f'{filename}.png', f'{filename}.mp3'

        cv.imwrite(frame_filename, frame)
        write_mp3(audio_filename, audio_rate, audio_interval)


In [12]:
fc = files_couple('VideoMP4/LaLinea/')
fc

[['VideoMP4/LaLinea/La Linea 131.mp4', 'VideoMP4/LaLinea/La Linea 131.mp3'],
 ['VideoMP4/LaLinea/La Linea 132.mp4', 'VideoMP4/LaLinea/La Linea 132.mp3']]

In [13]:
_ = [frame_sounds_couple(*c) for c in fc[1:2]]

c:\Users\tommaso\AppData\Local\Programs\Python\Python39\lib\site-packages\librosa\util\decorators.py:88: UserWarning: PySoundFile failed. Trying audioread instead.
  return f(*args, **kwargs)



# Load segments

In [168]:
def load_path_dataset(folder: str, window_size: int = 2):
    assert(window_size > 1)
    
    audiofiles = audio_from_folder(folder)
    imagefiles = image_from_folder(folder)
    assert(len(audiofiles)==len(imagefiles))

    audiofiles_windows = []
    for i in range(window_size-1, len(imagefiles)):
        audiofiles_window = []
        for p in range(window_size):
            audiofiles_window.append(audiofiles[i-p])   
        audiofiles_window.reverse() # o no ?
        audiofiles_windows.append(audiofiles_window)
    
    return [[audiofiles_window, image] for audiofiles_window, image in zip(audiofiles_windows, imagefiles[window_size-1:])]

def load_dataset_item(path_dataset: list, item_index: int):
    assert(item_index < len(path_dataset))
    path_dataset_item = path_dataset[item_index]
    dataset_item = [[librosa.load(af) for af in path_dataset_item[0]], imread(path_dataset_item[1])]
    return dataset_item

In [177]:
# RAM overload

# path_dataset = load_path_dataset('VideoMP4/SillySimphoniesSkeletons/Betty Boops Halloween Party 1933 720p/', 5)
# dataset_item = load_dataset_item(path_dataset, 3000)

In [14]:
x1, y1 = librosa.load('VideoMP4/LaLinea/La Linea 131/La Linea 131_seg_000001.mp3')
x2, y2 = librosa.load('VideoMP4/LaLinea/La Linea 132/La Linea 132_seg_000001.mp3')

c:\Users\tommaso\AppData\Local\Programs\Python\Python39\lib\site-packages\librosa\util\decorators.py:88: UserWarning: PySoundFile failed. Trying audioread instead.
  return f(*args, **kwargs)



In [63]:
np.concatenate([x1,x2]).shape[0] //2

882

# Tensorflow datagen

In [6]:
from tensorflow import keras
import pandas as pd

class EmilianDataGen(keras.utils.Sequence):

    # REF: [ https://medium.com/analytics-vidhya/write-your-own-custom-data-generator-for-tensorflow-keras-1252b64e41c3 ]
    
    def __init__(self, df, X_col, y_col, w_size, # Two columns dataset, X for audio paths (ap), y for frames path (fp), df will converted according to w_size -> item: [[ap_1, ap_2, ap_wsize], fp_wsize]
                 batch_size,
                 input_size,
                 shuffle=False):
        
        self.df = df.copy()
        self.X_col = X_col
        self.y_col = y_col
        self.batch_size = batch_size
        self.input_size = input_size
        self.w_size = w_size
        self.shuffle = shuffle

        self.format_wdf()

        self.n = len(self.df)

    def format_wdf(self):
        entries = []
        for i in range(len(self.df)):
            if i >= self.w_size-1:
                x = []
                for k in range(self.w_size):
                    x.append(self.df.iloc[i-k][self.X_col])
                x = list(reversed(x))
                y = self.df.iloc[i][self.y_col]
                entries.append([x, y])
        self.df = pd.DataFrame(entries, columns=[self.X_col, self.y_col])

    def __get_input(self, audio_paths):
        audios = []
        for p in audio_paths:
            a, _ = librosa.load(p)
            audios.append(a)
        return np.concatenate(audios)
    
    def __get_output(self, frame_path):
        image = keras.preprocessing.image.load_img(frame_path)
        image_arr = keras.preprocessing.image.img_to_array(image)
        return image_arr / 255.
    
    def __get_data(self, batches):
        # Generates data containing batch_size samples

        X_batch = np.asarray([self.__get_input(audio_paths) for audio_paths in batches[self.X_col].to_list()])
        y_batch = np.asarray([self.__get_output(frame_path) for frame_path in batches[self.y_col].to_list()])

        return X_batch, y_batch
    
    def __getitem__(self, index):
        batches = self.df[index*self.batch_size : (index+1)*self.batch_size]
        X, y = self.__get_data(batches)        
        return X, y
    
    def on_epoch_end(self):
        if self.shuffle:
            self.df = self.df.sample(frac=1).reset_index(drop=True)
    
    def __len__(self):
        return self.n // self.batch_size

In [7]:
audios = audio_from_folder('VideoMP4/LaLinea/La Linea 131/')
frames = image_from_folder('VideoMP4/LaLinea/La Linea 131/')
df = pd.DataFrame([[a,f] for a,f in zip(audios, frames)], columns=['audio', 'frame'])

In [8]:
edg = EmilianDataGen(df, 'audio', 'frame', 3, 16, 882)

In [9]:
items = edg.__getitem__(1)

print('len items (2, one for audio_windows one for frames)      ---->   ', len(items))
print('shape input (audio) = (batch_size, audio_size*w_size)    ---->   ', items[0].shape)
print('shape output (frame) = (batch_size, im_h, im_w, n_ch)    ---->   ', items[1].shape)

c:\Users\tommaso\AppData\Local\Programs\Python\Python39\lib\site-packages\librosa\util\decorators.py:88: UserWarning: PySoundFile failed. Trying audioread instead.
  return f(*args, **kwargs)



len items (2, one for audio_windows one for frames)      ---->    2
shape input (audio) = (batch_size, audio_size*w_size)    ---->    (16, 2646)
shape output (frame) = (batch_size, im_h, im_w, n_ch)    ---->    (16, 360, 472, 3)


In [ ]:
# Use in a tf.keras model

# model.fit(edg,
#           validation_size=0.1,
#           epochs=10)